In [ ]:
#hide
from ModelAssistedLabel.core import *

# YOLOv5 Model Assisted Labeling

> Work based on https://models.roboflow.com/object-detection/yolov5.

Labeling a new dataset from scratch is tiresome. To label images faster, I train a YOLOv5 model and apply its predictions to new images.

## Install

`pip install your_project_name`

## How to use

### Vanilla image sets

In [ ]:
%%capture
#hide
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g7tGbDc1dmqMisGRCMTBZ-afkJknMexgtpAYx9iFpZaoD3J3G2AScw


In [ ]:
#hide
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"
%run '_Synch.ipynb'

/content/drive/MyDrive/Coding/ModelAssistedLabel
updating index
Converted 00_ultralytics.ipynb.
Converted 01_split.ipynb.
Converted 02_augment.ipynb.
Converted index.ipynb.
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/index.ipynb
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/01_split.ipynb
converting /content/drive/My Drive/Coding/ModelAssistedLabel/index.ipynb to README.md
Executing: git config --local include.path ../.gitconfig
Success: hooks are installed and repo's .gitconfig is now trusted


In [ ]:
ModelAssistedLabel